In [28]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random

import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings(action='ignore')

In [6]:
# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)

In [8]:
PATH = './data/'

train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [9]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [10]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [11]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:44<00:00, 372564.92it/s]


(15000, 42)

In [ ]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

In [12]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [13]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = error
train_y = problem
del error, problem
print(train_x.shape)
print(train_y.shape)

(15000, 42)
(15000,)


In [14]:
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True

In [15]:
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }

In [32]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

Training until validation scores don't improve for 3 rounds
[20]	valid_0's auc: 0.797673	valid_0's pr_auc: 0.792522
Early stopping, best iteration is:
[25]	valid_0's auc: 0.801171	valid_0's pr_auc: 0.797338
!!!!!!!valid_prob  [0.16004703 0.06651394 0.85864387 ... 0.12651282 0.18023637 0.17748836]
!!!!!!!valid_pred  [0 0 1 ... 0 0 0]
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.788479	valid_0's pr_auc: 0.318902
!!!!!!!valid_prob  [0.38726621 0.38949887 0.36214486 ... 0.40558827 0.36747112 0.41369381]
!!!!!!!valid_pred  [0 0 0 ... 0 0 0]
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.758498	valid_0's pr_auc: 0.358092
!!!!!!!valid_prob  [0.36350082 0.35522081 0.34904372 ... 0.33821435 0.33821435 0.34904372]
!!!!!!!valid_pred  [0 0 0 ... 0 0 0]
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[10]	valid_0's

In [17]:
print(np.mean(auc_scores))

0.7915834292563759


In [18]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [19]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [20]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:48<00:00, 342980.72it/s]

(14999, 42)


In [33]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    print(pred_y)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

[0.77693909 0.16723148 0.25590885 ... 0.36093974 0.74180179 0.31111278]
[0.48348078 0.37603432 0.38309442 ... 0.45845688 0.50092646 0.43057709]
[0.46785387 0.36350082 0.38727352 ... 0.42835311 0.49270365 0.39992439]
[0.67214765 0.2724766  0.39263204 ... 0.41876583 0.72163114 0.37781292]
[0.52026454 0.13763469 0.15790022 ... 0.2349051  0.62078147 0.15763059]
[0.77693909 0.16723148 0.25590885 ... 0.36093974 0.74180179 0.31111278]
[0.48348078 0.37603432 0.38309442 ... 0.45845688 0.50092646 0.43057709]
[0.46785387 0.36350082 0.38727352 ... 0.42835311 0.49270365 0.39992439]
[0.67214765 0.2724766  0.39263204 ... 0.41876583 0.72163114 0.37781292]
[0.52026454 0.13763469 0.15790022 ... 0.2349051  0.62078147 0.15763059]
[0.77693909 0.16723148 0.25590885 ... 0.36093974 0.74180179 0.31111278]
[0.48348078 0.37603432 0.38309442 ... 0.45845688 0.50092646 0.43057709]
[0.46785387 0.36350082 0.38727352 ... 0.42835311 0.49270365 0.39992439]
[0.67214765 0.2724766  0.39263204 ... 0.41876583 0.72163114 0.37

In [29]:
pred_y_list

[array([[0.77693909],
        [0.16723148],
        [0.25590885],
        ...,
        [0.36093974],
        [0.74180179],
        [0.31111278]]),
 array([[0.48348078],
        [0.37603432],
        [0.38309442],
        ...,
        [0.45845688],
        [0.50092646],
        [0.43057709]]),
 array([[0.46785387],
        [0.36350082],
        [0.38727352],
        ...,
        [0.42835311],
        [0.49270365],
        [0.39992439]]),
 array([[0.67214765],
        [0.2724766 ],
        [0.39263204],
        ...,
        [0.41876583],
        [0.72163114],
        [0.37781292]]),
 array([[0.52026454],
        [0.13763469],
        [0.15790022],
        ...,
        [0.2349051 ],
        [0.62078147],
        [0.15763059]])]

In [22]:
pred_ensemble

array([[0.58413718],
       [0.26337558],
       [0.31536181],
       ...,
       [0.38028413],
       [0.6155689 ],
       [0.33541155]])

In [25]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [26]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [27]:
sample_submssion.to_csv("./submission/submission_20210114-0.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.584137
1,30001,0.263376
2,30002,0.315362
3,30003,0.539370
4,30004,0.464485
...,...,...
14994,44994,0.320964
14995,44995,0.320746
14996,44996,0.380284
14997,44997,0.615569
